# Preprocessing
## Imports

In [36]:
import networkx as nx
from langchain_core.documents import Document
import pandas as pd
import json
import re

## Functions

In [136]:
def get_base_id(id: str) -> str:
    match = re.match(r'\b(?:Article|Art\.|Rule|R\.)\s\d+[a-zA-Z]?\b', id)
    return match.group()

In [137]:
def load_data(path: str) -> list:
    with open(path, 'r') as js:
        data = json.load(js)
    return data

In [138]:
def create_doc(row: pd.Series) -> Document:
    doc = Document(page_content=row["text"], id=row["title"], metadata=row.drop(labels=["text", "title"]).to_dict())
    return doc

In [139]:
def add_to_graph(df: pd.DataFrame, g: nx.MultiGraph, label: str):
    for _, row in df.iterrows():
        doc = create_doc(row)
        if "implementation" in df.columns:
            id = doc.id
        else:
            id = get_base_id(doc.id)
        g.add_node(id, data=doc, label=label)

In [168]:
def connect(df: pd.DataFrame, g: nx.MultiGraph):
    if "implementation" in df.columns:
        for _, row in df.iterrows():
            if isinstance(row["implementation"], dict):
                node1_id = row["title"]
                implementations = row["implementation"]
                for implementation in implementations.keys():
                    if get_base_id(implementation) in g:
                        g.add_edge(node1_id, get_base_id(implementation), key="implements")
    else:
        for _, row in df.iterrows():
            node1_id = get_base_id(row["title"])
            references = row["references"]
            for ref in references:
                if get_base_id(ref) in g:
                    g.add_edge(node1_id, get_base_id(ref), key="references")

In [141]:
def clean_list(lst):
    if isinstance(lst, list):
        return [s.replace('\xa0', ' ').strip() if isinstance(s, str) else s for s in lst]
    return lst

In [142]:
def clean_dict(dct):
    VALID_KEY_PATTERN = re.compile(r'^(?:Article|Art\.|Rule|R\.)\s\d+[a-zA-Z]?$')
    if isinstance(dct, dict):
        cleaned = {}
        for k, v in dct.items():
            clean_key = k.replace('\xa0', ' ').strip()
            if VALID_KEY_PATTERN.match(clean_key):
                clean_value = [w.replace('\xa0', ' ').strip() for w in v]
                cleaned[clean_key] = clean_value
        return cleaned
    return dct

In [143]:
def get_text(dct):
    if isinstance(dct, dict):
        flat = []
        for v in dct.values():
            if isinstance(v, list):
                flat.extend(str(item) for item in v)
            else:
                flat.append(str(v))
        return " ".join(flat)
    return dct

## Main

### EPC

In [144]:
path = "../../resources/extracted_data/EPC_data.json"
articles = load_data(path)
articles_df = pd.DataFrame(articles)
articles_df = articles_df.replace(u"\xa0", ' ', regex=True)
articles_df = articles_df.map(lambda x: x.strip() if isinstance(x, str) else x)
articles_df['references'] = articles_df['references'].apply(clean_list)
articles_df

,book,part,chapter,title,text,references,url
0,EPC,Part I - General and institutional provisions,Chapter I - General provisions,Article 1 - European law for the grant of patents,"A system of law, common to the Contracting Sta...",,https://www.epo.org/en/legal/epc/2020/a1.html
1,EPC,Part I - General and institutional provisions,Chapter I - General provisions,Article 2 - European patent,(1) Patents granted under this Convention shal...,"[Article 63 Term of the European patent, Artic...",https://www.epo.org/en/legal/epc/2020/a2.html
2,EPC,Part I - General and institutional provisions,Chapter I - General provisions,Article 3 - Territorial effect,The grant of a European patent may be requeste...,"[Article 79 Designation of Contracting States,...",https://www.epo.org/en/legal/epc/2020/a3.html
3,EPC,Part I - General and institutional provisions,Chapter I - General provisions,Article 4 - European Patent Organisation,"(1) A European Patent Organisation, hereinafte...",[Article 4a Conference of ministers of the Con...,https://www.epo.org/en/legal/epc/2020/a4.html
4,EPC,Part I - General and institutional provisions,Chapter I - General provisions,Article 4a - Conference of ministers of the Co...,A conference of ministers of the Contracting S...,[Article 4 European Patent Organisation],https://www.epo.org/en/legal/epc/2020/a4a.html
...,...,...,...,...,...,...,...
163,EPC,Part XII - Final provisions,Art. 167 - (deleted),Article 174 - Denunciation,Any Contracting State may at any time denounce...,"[Article 175 Preservation of acquired rights, ...",https://www.epo.org/en/legal/epc/2020/a174.html
164,EPC,Part XII - Final provisions,Art. 167 - (deleted),Article 175 - Preservation of acquired rights,(1) In the event of a State ceasing to be part...,,https://www.epo.org/en/legal/epc/2020/a175.html
165,EPC,Part XII - Final provisions,Art. 167 - (deleted),Article 176 - Financial rights and obligations...,(1) Any State which has ceased to be a party t...,,https://www.epo.org/en/legal/epc/2020/a176.html
166,EPC,Part XII - Final provisions,Art. 167 - (deleted),Article 177 - Languages of the Convention,"(1) This Convention, drawn up in a single orig...",,https://www.epo.org/en/legal/epc/2020/a177.html


### EPC regulations


In [145]:
path = "../../resources/extracted_data/EPC_regulations.json"
rules = load_data(path)
rules_df = pd.DataFrame(rules)
rules_df = rules_df.replace(u"\xa0", ' ', regex=True)
rules_df = rules_df.map(lambda x: x.strip() if isinstance(x, str) else x)
rules_df['references'] = rules_df['references'].apply(clean_list)
rules_df

,book,part,chapter,section,title,text,references,url
0,EPC,Part I - Implementing Regulations to part I of...,Chapter I - General provisions,,Rule 1 - Written proceedings,In written proceedings before the European Pat...,"[Rule 3 Language in written proceedings, Rule ...",https://www.epo.org/en/legal/epc/2020/r1.html
1,EPC,Part I - Implementing Regulations to part I of...,Chapter I - General provisions,,Rule 2 - Filing of and formal requirements for...,(1)3 In proceedings before the European Patent...,"[Rule 3 Language in written proceedings, Rule ...",https://www.epo.org/en/legal/epc/2020/r2.html
2,EPC,Part I - Implementing Regulations to part I of...,Chapter I - General provisions,,Rule 3 - Language in written proceedings,(1) In written proceedings before the European...,[Article 14 Languages of the European Patent O...,https://www.epo.org/en/legal/epc/2020/r3.html
3,EPC,Part I - Implementing Regulations to part I of...,Chapter I - General provisions,,Rule 4 - Language in oral proceedings,(1) Any party to oral proceedings before the E...,[Article 14 Languages of the European Patent O...,https://www.epo.org/en/legal/epc/2020/r4.html
4,EPC,Part I - Implementing Regulations to part I of...,Chapter I - General provisions,,Rule 5 - Certification of translations,Where the translation of a document is require...,[Article 14 Languages of the European Patent O...,https://www.epo.org/en/legal/epc/2020/r5.html
...,...,...,...,...,...,...,...,...
171,EPC,Part IX - Implementing Regulations to Part X o...,,,Rule 161 - Amendment of the application,(1) If the European Patent Office has acted as...,"[Article 123 Amendments, Article 152 The Europ...",https://www.epo.org/en/legal/epc/2020/r161.html
172,EPC,Part IX - Implementing Regulations to Part X o...,,,Rule 162 - Claims incurring fees,(1) If the application documents on which the ...,[Article 153 The European Patent Office as des...,https://www.epo.org/en/legal/epc/2020/r162.html
173,EPC,Part IX - Implementing Regulations to Part X o...,,,Rule 163 - Examination of certain formal requi...,(1) Where the designation of the inventor unde...,"[Article 81 Designation of the inventor, Artic...",https://www.epo.org/en/legal/epc/2020/r163.html
174,EPC,Part IX - Implementing Regulations to Part X o...,,,Rule 164 - Unity of invention and further sear...,(1) If the European Patent Office considers th...,"[Article 82 Unity of invention, Article 153 Th...",https://www.epo.org/en/legal/epc/2020/r164.html


### EPO guidelines

In [146]:
path = "../../resources/extracted_data/EPO_guidelines.json"
guidelines = load_data(path)
guidelines_df = pd.DataFrame(guidelines)
guidelines_df = guidelines_df.replace(u"\xa0", ' ', regex=True)
guidelines_df = guidelines_df.map(lambda x: x.strip() if isinstance(x, str) else x)
guidelines_df['implementation'] = guidelines_df['implementation'].apply(clean_dict)
guidelines_df["text"] = guidelines_df["implementation"].apply(get_text)
guidelines_df

,book,part,chapter,title,implementation,text
0,EPO_guidelines,Part A – Guidelines for Formalities Examination,Chapter I – Introduction,Chapter I – Introduction,,
1,EPO_guidelines,Part A – Guidelines for Formalities Examination,Chapter I – Introduction,1. Overview,Part A of the Guidelines deals with:\n(i)the r...,Part A of the Guidelines deals with:\n(i)the r...
2,EPO_guidelines,Part A – Guidelines for Formalities Examination,Chapter I – Introduction,2. Responsibility for formalities examination,{'Rule 10': ['The matters covered in this part...,The matters covered in this part are intended ...
3,EPO_guidelines,Part A – Guidelines for Formalities Examination,Chapter I – Introduction,3. Purpose of Part A,This part of the Guidelines is intended to pro...,This part of the Guidelines is intended to pro...
4,EPO_guidelines,Part A – Guidelines for Formalities Examination,Chapter I – Introduction,4. Other parts relating to formalities,Formalities officers should not concern themse...,Formalities officers should not concern themse...
...,...,...,...,...,...,...
1779,EPO_guidelines,Part H – Amendments and Corrections,Chapter VI – Correction of errors,3.2 Allowability of the correction of bibliogr...,The sole reason for allowing the correction of...,The sole reason for allowing the correction of...
1780,EPO_guidelines,Part H – Amendments and Corrections,Chapter VI – Correction of errors,3.3 Correction of the decision to grant while ...,"Even during opposition proceedings, the examin...","Even during opposition proceedings, the examin..."
1781,EPO_guidelines,Part H – Amendments and Corrections,Chapter VI – Correction of errors,4. Correction of formatting/editing errors,Formatting/editing errors which were already c...,Formatting/editing errors which were already c...
1782,EPO_guidelines,Part H – Amendments and Corrections,Chapter VI – Correction of errors,5. Correction of the translations of the claims,"According to Art. 70(1), the text of a patent ...","According to Art. 70(1), the text of a patent ..."


In [169]:
g = nx.MultiGraph()

In [170]:
add_to_graph(articles_df, g, "Article")
add_to_graph(rules_df, g, "Rule")
add_to_graph(guidelines_df, g, "Guideline")
connect(articles_df, g)
connect(rules_df, g)
connect(guidelines_df, g)

In [ ]:
import matplotlib.pyplot as plt


# Get labels from node attributes
labels = nx.get_node_attributes(g, 'label')

# Map labels to unique colors
unique_labels = list(set(labels.values()))
label_color_map = {label: i for i, label in enumerate(unique_labels)}

# Assign a color to each node based on its label
node_colors = [label_color_map[labels[node]] for node in g.nodes()]

# Draw graph without labels, but with colors
pos = nx.spring_layout(g)
nx.draw(g, pos, node_color=node_colors, with_labels=False, cmap=plt.cm.Set3, node_size=10)

plt.show()

In [14]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_core.documents import Document

local_embeddings = OllamaEmbeddings(model="all-minilm")

vectorstore = Chroma.from_texts(articles_df["text"].to_list(), embedding=local_embeddings)

""" config_path = "../../config/retriever_config.yaml"
retriever = Retriever(config_path, articles_df) """

' config_path = "../../config/retriever_config.yaml"\nretriever = Retriever(config_path, articles_df) '

In [15]:
query = "How is decided whether an idea is novel or not?"
docs = vectorstore.similarity_search(query)

In [16]:
docs

[Document(id='af79404e-38c6-49c4-a85f-36f0ce4f2256', metadata={}, page_content='An invention shall be considered as involving an inventive step if, having regard to the state of the art, it is not obvious to a person skilled in the art. If the state of the art also includes documents within the meaning of Article 54, paragraph 3, these documents shall not be considered in deciding whether there has been an inventive step.\n\n\n46See decisions/opinions of the Enlarged Board of Appeal G 2/98, G 3/98, G 2/99, G 1/03, G 2/03, G 1/16 (Annex I).'),
 Document(id='7e329766-d354-4154-8dfe-f5e5e085e133', metadata={}, page_content='An invention shall be considered as involving an inventive step if, having regard to the state of the art, it is not obvious to a person skilled in the art. If the state of the art also includes documents within the meaning of Article 54, paragraph 3, these documents shall not be considered in deciding whether there has been an inventive step.\n\n\n46See decisions/opin

In [19]:
g.edges("Article 63")

OutMultiEdgeDataView([('Article 2 - European patent', 'Article 63 Term of the European patent'), ('Article 2 - European patent', 'Article 65 Translation of the European patent'), ('Article 2 - European patent', 'Article 68 Effect of revocation or limitation of the European patent'), ('Article 2 - European patent', 'Article 69 Extent of protection'), ('Article 2 - European patent', 'Article 70 Authentic text of a European patent application or European patent'), ('Article 2 - European patent', 'Article 99 Opposition'), ('Article 2 - European patent', 'Article 105c Publication of the amended specification of the European patent'), ('Article 2 - European patent', 'Article 142 Unitary patents'), ('Article 2 - European patent', 'Rule 75 Surrender or lapse of the patent'), ('Article 2 - European patent', 'Rule 85 Transfer of the European patent'), ('Article 2 - European patent', 'Rule 89 Intervention of the assumed infringer'), ('Article 2 - European patent', 'Rule 90 Subject of proceedings'